In [1]:
import numpy as np
import pandas as pd

In [2]:
dic={'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':float, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [3]:
ls

Overfitting_Demo_Ridge_Lasso.ipynb  week5 assignments 2.ipynb
kc_house_data.csv*                  wk3_kc_house_test_data.csv*
kc_house_test_data.csv*             wk3_kc_house_train_data.csv*
kc_house_train_data.csv*            wk3_kc_house_valid_data.csv*
week5 assignments 1.ipynb


In [4]:
alldata=pd.read_csv('kc_house_data.csv',dtype=dic)
train=pd.read_csv('kc_house_train_data.csv',dtype=dic)
test=pd.read_csv('kc_house_test_data.csv',dtype=dic)

In [5]:
def get_numpy_data(data,features,output):
    data['constant'] = 1
    features = ['constant'] + features
    featuredata=data[features]
    features_matrix=featuredata.values
    outdata=data[output]
    output_array =outdata.values
    return(features_matrix, output_array)

In [6]:
def predict_output(feature_matrix, weights):               # predict outcomes
    predictions=np.dot(feature_matrix, weights)
    return predictions

In [7]:
def normalize_features(features):
    norms=np.linalg.norm(features,axis=0)
    normalized_features=features/norms
    return (normalized_features, norms)

In [8]:
iniw=np.array([1.,4.,1.])
fe=['sqft_living','bedrooms']
ou='price'
(fema,output)=get_numpy_data(alldata,fe,ou)
feman=normalize_features(fema)[0]
predict=predict_output(feman,iniw)

In [9]:
ro=np.dot(feman[:,1],output-predict+iniw[1]*feman[:,1])
ro*2

175878941.64650351

In [10]:
ro=np.dot(feman[:,2],output-predict+iniw[2]*feman[:,2])
ro*2

161933397.33247894

In [11]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction =predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.dot(feature_matrix[:,i],output-prediction+weights[i]*feature_matrix[:,i])
    #print ro_i
    # this is very important: if i want i column,choose feature_matrix[:,i]
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i+l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i-l1_penalty/2.
    else:
        new_weight_i = 0.    
    return new_weight_i

In [12]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


In [13]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    converge=False
    weights=np.array(initial_weights)
    while converge==False:
        diff=[]
        for i in range(0,len(weights)):
            a=lasso_coordinate_descent_step(i,feature_matrix,output,weights,l1_penalty)
            diff.append(abs(a-weights[i]))
            weights[i]=a
        ll=np.array(diff)
        if (ll.max()<tol):
            converge=True
        #print weights
    return weights

### 1

In [14]:
fe1=['sqft_living','bedrooms']
ou1='price'
(fema1,output1)=get_numpy_data(train,fe1,ou1)

In [15]:
fema[:,0]

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [16]:
(feman1,norm1)=normalize_features(fema)

In [17]:
weights=np.array([0.,0.,0.])
l1=1e7
tol=1.0

In [18]:
we1=lasso_cyclical_coordinate_descent(feman1, output, weights, l1, tol)

In [19]:
np.sum((output-predict_output(feman,we1))**2)

1630492476715387.0

## 2

In [20]:
fe2=['bedrooms', 'bathrooms','sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement','yr_built', 'yr_renovated'
]
ou2='price'

In [21]:
(fema2,output2)=get_numpy_data(train,fe2,ou2)

In [22]:
fema2[:,0]

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

In [23]:
(feman2,norm2)=normalize_features(fema2)

In [24]:
feman2

array([[ 0.00758447,  0.00652117,  0.0033687 , ...,  0.        ,
         0.00752148,  0.        ],
       [ 0.00758447,  0.00652117,  0.00757957, ...,  0.0057043 ,
         0.0075061 ,  0.03707954],
       [ 0.00758447,  0.00434745,  0.0033687 , ...,  0.        ,
         0.00743684,  0.        ],
       ..., 
       [ 0.00758447,  0.00652117,  0.00842175, ...,  0.        ,
         0.00772924,  0.        ],
       [ 0.00758447,  0.00652117,  0.00842175, ...,  0.        ,
         0.00771   ,  0.        ],
       [ 0.00758447,  0.00434745,  0.00252652, ...,  0.        ,
         0.00772539,  0.        ]])

In [25]:
we=np.linspace(0.0,0.0,14)
l1=1e7
tol=1.0
we1e7=lasso_cyclical_coordinate_descent(feman2, output2, we, l1, tol)

In [26]:
we=np.linspace(0.0,0.0,14)
l1=1e8
tol=1.0
we1e8=lasso_cyclical_coordinate_descent(feman2, output2, we, l1, tol)

In [27]:
we=np.linspace(0.0,0.0,14)
l1=1e4
tol=5e5
we1e4=lasso_cyclical_coordinate_descent(feman2, output2, we, l1, tol)

In [28]:
fex=['in']+fe2

In [29]:
for i in range(0,len(fex)):
    if( we1e7[i]!=0):
        print fex[i]

in
sqft_living
waterfront
view


In [30]:
for i in range(0,len(fex)):
    if( we1e8[i]!=0):
        print fex[i]

in


In [31]:
for i in range(0,len(fex)):
    if( we1e4[i]!=0):
        print fex[i]

in
bedrooms
bathrooms
sqft_living
sqft_lot
floors
waterfront
view
condition
grade
sqft_above
sqft_basement
yr_built
yr_renovated


In [33]:
we1=we1e7/norm2
we2=we1e8/norm2
we3=we1e4/norm2

## very important, need to correct our coeffs

In [34]:
fe=['sqft_living','bedrooms']
ou='price'
(fema,output)=get_numpy_data(test,fe,ou)

In [35]:
fe2=['bedrooms', 'bathrooms','sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement','yr_built', 'yr_renovated'
]
ou2='price'
(fema2,output2)=get_numpy_data(test,fe2,ou2)

In [36]:
rss1=np.sum((output2-predict_output(fema2,we1))**2)
rss2=np.sum((output2-predict_output(fema2,we2))**2)
rss3=np.sum((output2-predict_output(fema2,we3))**2)

In [37]:
print rss1,rss2,rss3

2.7596207592e+14 5.37166151497e+14 2.28459958971e+14
